###  1 - Carregar os pactotes e configurações do ambiente python

In [13]:
import os.path as osp, os
os.environ["CUDA_VISIBLE_DEVICES"] = ''
import torch, numpy as np
from data.graphdataset import ThermoMLDataset, ramirez, ThermoMLpara
from train.train import create_model
from train.models import PNAPCSAFT
from train.model_deg import calc_deg
from data.graph import from_InChI
from train.parametrisation import MAPE, rhovp_data
import matplotlib.pyplot as plt
from configs.default import get_config
config = get_config()
device = torch.device("cpu")
model_dtype = torch.float64
deg_model2 = calc_deg("thermoml", './')
deg_model1 = calc_deg("ramirez", './')
from demo.utils import loadckp, plotdata
print(torch.cuda.is_available())
import networkx as nx 
from torch_geometric.utils import to_networkx
from torch_geometric.nn import summary

### 2 - Carregar os dados de parametrização e dados experimentais de densidade e pressão de vapor

* Dados de parametrização

In [14]:
ra_loader = ramirez("./data/ramirez2022")
ra_para = {}
for graph in ra_loader:
    inchi, para = graph.InChI, graph.para.view(-1, 3).round(decimals=2)
    ra_para[inchi] = para.tolist()[0]

In [15]:
tml_loader = ThermoMLpara("./data/thermoml")
tml_para = {}
for graph in tml_loader:
    inchi, para = graph.InChI, graph.para.view(-1, 3).round(decimals=2)
    tml_para[inchi] = para.tolist()[0]

* Dados de pressão de vapor e densidade 

In [16]:
path = osp.join("data", "thermoml")
testloader = ThermoMLDataset(path)


### 3 - Criar e inicializar a Graph Neural Network


In [17]:
config.propagation_depth=4
config.hidden_dim=128
config.num_mlp_layers=2
config.pre_layers=1
config.post_layers=3

In [18]:
model2 = create_model(config, deg_model2).to(device, model_dtype).eval()
model1 = create_model(config, deg_model1).to(device, model_dtype).eval()

* Resumo dos parâmetros do modelo

In [19]:
dummy = tml_loader[0]
dummy.x = dummy.x.to(model_dtype)
dummy.edge_attr = dummy.edge_attr.to(model_dtype)
dummy.edge_index = dummy.edge_index.to(torch.int64)
print(summary(model2, dummy, max_depth=2))

* Carregar os parâmetros dos modelos 1 e 2 treinados que foram salvos

In [20]:
ckp_path = "./train/checkpoints/model2-13_62e6.pth"
loadckp(ckp_path, model2)
ckp_path = "./train/checkpoints/model1-9_20e6.pth"
loadckp(ckp_path, model1)

### 4 - Fazer a predição dos parâmetros a partir dos grafos

* InChI da molécula

In [21]:
inchi = "InChI=1S/C5H6O2/c6-4-5-2-1-3-7-5/h1-3,6H,4H2"
#inchi = "InChI=1S/H2O/h1H2"

* Grafo da molécula

In [25]:
grafo = from_InChI(inchi).to(device)
print("###### Nós (átomos) do grafo e seus atributos ###### \n")
print(grafo.x)
print("\n###### Arestas (ligações químicas) da molécula e seus atributos ######\n")
print(grafo.edge_attr)
print("\n###### Figura do grafo ######\n")
nxg = to_networkx(grafo)
nx.draw(nxg)

* Parâmetros previstos pelos Modelo 1 e 2

In [29]:
grafo.x = grafo.x.to(model_dtype)
grafo.edge_attr = grafo.edge_attr.to(model_dtype)
grafo.edge_index = grafo.edge_index.to(torch.int64)
with torch.no_grad():
    parametros_modelo_1 = model1(grafo)
    parametros_modelo_2 = model2(grafo)
print("######### Parâmetros previstos #########\n")
print("Pelo Modelo 1:\n")
print("[número de segmentos, diâmetros de segmento (Å), energia de dispersão (K)]")
print(parametros_modelo_1[0].numpy().round(4))
print("\nPelo Modelo 2:\n")
print("[número de segmentos, diâmetros de segmento (Å), energia de dispersão (K)]")
print(parametros_modelo_2[0].numpy().round(4))

* Figuras demonstrando o desempenho dos parâmetros

In [30]:
inchi = inchi
molecule_name = "furfuranol"
print(f"######### {molecule_name} #########")
plotdata(inchi, molecule_name, models=[model1, model2])
